In [20]:
import os
import datetime
import holidays
import pandas as pd

In [21]:
PL_HOLIDAYS = holidays.country_holidays('PL')
LIFE_EXPECTANCY = 80

def calculate_free_days(years_from_now = LIFE_EXPECTANCY) -> pd.DataFrame:
    weekend_count = 0
    year = datetime.datetime.now().year
    log = {
        'year': [],
        'weekend_count': [],
        'if_leap': [],
    }

    for n in range(366 * years_from_now):
        cur_date = datetime.datetime.now() + datetime.timedelta(n)
        leap = False
        if cur_date.day == 29 and cur_date.month == 2:
            leap = True
        if cur_date.year != year:
            year = cur_date.year
            log['year'].append(year)
            log['weekend_count'].append(weekend_count)
            log['if_leap'].append(leap)
            weekend_count = 0

        if cur_date.weekday() in [5, 6] or cur_date in PL_HOLIDAYS:
            weekend_count += 1
        
    log_df = pd.DataFrame(log)
    return log_df

In [22]:
# Calculating best year of life (depending on work free time)

log_df = calculate_free_days(LIFE_EXPECTANCY)

log_df['best_year'] = log_df['weekend_count'].apply(lambda x: True if x == log_df['weekend_count'].max() else False)
if not os.path.isfile('free_time.xlsx') or print('File already exists (delete/rename before saving)'):
    log_df.to_excel('free_time.xlsx')
    
log_df.sort_values(by='best_year', ascending=False)

File already exists (delete/rename before saving)


,year,weekend_count,if_leap,best_year
32,2057,115,False,True
4,2029,115,False,True
60,2085,115,False,True
51,2076,114,False,False
58,2083,111,False,False
...,...,...,...,...
26,2051,113,False,False
25,2050,111,False,False
24,2049,111,False,False
23,2048,114,False,False


In [23]:
# Calculating best year for person to be born (depending on all work/shool free time available)

BORN_TIME_PERIOD = 60 # given that you are be born in next 60 years
year_to_check = (BORN_TIME_PERIOD + LIFE_EXPECTANCY) + 10 # and live LIFE_EXPECTANCY years

log_df = calculate_free_days(year_to_check)

best_year_to_born = log_df['weekend_count'].rolling(LIFE_EXPECTANCY).sum().dropna()#
best_year_to_born = pd.DataFrame(best_year_to_born)
best_year_to_born.axes[0].name = 'year'
best_year_to_born = best_year_to_born.reset_index()
best_year_to_born['year'] = best_year_to_born['year']  - LIFE_EXPECTANCY + 1 + 2025
best_year_to_born = best_year_to_born.sort_values(by='weekend_count', ascending=False)
best_year_to_born = best_year_to_born.rename({'weekend_count': 'LIFE_ALL_FREE_DAYS'}, axis=1)
best_year_to_born.head(20)

,year,LIFE_ALL_FREE_DAYS
55,2080,9022.0
45,2070,9022.0
44,2069,9022.0
27,2052,9022.0
17,2042,9022.0
16,2041,9022.0
28,2053,9021.0
15,2040,9021.0
54,2079,9021.0
46,2071,9021.0
